## Adquire Layout

Visit: [EK Seating Charts](https://www.emirates.com/ae/english/experience/seating-charts/)

Use this JavaScript function to manually donwload all SVG layouts one by one

```javascript
function hackLayout(){
    var svg = document.getElementsByTagName('svg')[0];
    var filename = ['data-aircraft', 'data-three-d-config', 'data-deck'].map(attr => svg.getAttribute(attr)).join('-') + '.svg';
    
    var a = document.createElement('a');
    a.style.display = 'none';

    a.setAttribute('href', 'data:text/plain;charset=utf-8,' + encodeURIComponent( svg.outerHTML ));
    a.setAttribute('download', filename);
        
    document.body.appendChild(a);
    a.click();
    document.body.removeChild(a);
}
```

## Use layout as heatmap

In [1]:
import numpy as np
import base64

class EKSeatingHeatmap():

    def __init__(self, svg_layout, scale_factor=0.5, cmin='#FFFFFF', cmax='#1E88E5'):

        with open(svg_layout, 'r') as file:
            self.svg_content = file.read()
            self.svg_content_with_seats = ''

        self.scale_factor = scale_factor
        self.cmap = self.generate_palette(cmin, cmax)

    def generate_palette(self, cmin, cmax, steps=100):

        def to_rgb(hex):
            return [int(hex[i:i+2], 16) for i in range(1,6,2)]

        def to_hex(rgb):
            return '#' + ''.join([
                '0{0:x}'.format(c) if c < 16 else '{0:x}'.format(c) for c in rgb
            ])

        cstart = to_rgb( cmin )
        cfinal = to_rgb( cmax )
        cdelta = [ cfinal[k] - cstart[k] for k in range(3) ]

        steps-= 1
        palette = [ cmin ]
        
        for step in range(1, steps):
            portion = step / steps
            cstep = [ int( cstart[k] + ( portion * cdelta[k] ) ) for k in range(3) ]
            palette.append( to_hex( cstep ) )

        palette.append( cmax )
        
        return palette

    def set_seats(self, seating_map, vmin=0, vmax=None):

        # Scale Values
        values = np.array( list( seating_map.values() ) )
        
        if vmin is None:
            vmin = values.min()

        if vmax is None:
            vmax = values.max()

        vrange = vmax - vmin

        values_scaled = list(map(
            lambda value: ( max(vmin, min(vmax, value)) - vmin ) / vrange,
            values
        ))

        self.data = list(zip(
            list( seating_map.keys() ),
            values_scaled,
            list( map(lambda value: self.cmap[ int(value * 100) - 1 ], values_scaled) )
        ))

        # Scale Layout
        svg_content = self.svg_content + ''

        size = tuple(map(lambda px: int(px * self.scale_factor), (6109, 5665)))
        svg_content = svg_content.replace('width="6109px" height="5665px"', 'width="%dpx" height="%spx"' % size)
        svg_content = svg_content.replace('font-size:13.2px', 'font-size:1.25em')

        # Map Colors
        seats_colored = []

        for seat_id, seat_value, seat_color in self.data:
            seats_colored.append(f'#\\3{ seat_id[0] } { seat_id[ 1 : ] } * rect {{ fill: { seat_color }; }}')

        # Add Color Map to SVG
        svg_content = svg_content.replace('</style>', f'{ "".join( seats_colored ) }</style>')

        # Save It
        self.svg_content_with_seats = svg_content
        
        return self

    def to_html_img(self):
        img_as_bytes = self.svg_content_with_seats.encode()
        img_base64_as_bytes = base64.b64encode( img_as_bytes )
        img_base64_as_str = img_base64_as_bytes.decode('utf-8')
        return f'<img src="data:image/svg+xml;base64,{ img_base64_as_str }"/>'

    def to_svg(self, filename = None):
        
        if filename is None:
            return self.svg_content_with_seats
        else:
            with open(filename, 'w') as file:
                file.write( self.svg_content_with_seats )
        
        return self
    
    def to_html(self, filename = None):

        html_content = f'''
            <html>
              <head>
                <title>Layout Heatmap</title>
                <style>
                    svg {{
                        border: 1px solid gray;
                        border-radius: 8px;
                        padding: 10px;
                    }}
                </style>
              </head>
              <body>
                { self.svg_content_with_seats }            
              </body>
            </html>
        '''

        if filename is None:
            return html_content
        else:
            with open(filename, 'w') as file:
                file.write( html_content )
        
        return self


In [2]:
seats = EKSeatingHeatmap('layouts/380-800-A380_CONFIG5-UD.svg')

seats.set_seats({
    '1A': 100,
    '1E': 90,
    '1F': 80,
    '1K': 70,
    '2A': 60,
    '2E': 50,
    '2F': 40,
    '2K': 30,
    '3A': 20,
    '3E': 10,
    '3F': 20,
    '3K': 30,
    '4A': 40,
    '4K': 50,
    '6D': 60,
    '6G': 70,
    '7A': 80,
    '7E': 90,
    '7F': 100
})


In [3]:
# Util to print html
from IPython.core.display import HTML
print_html = lambda html_content: display(HTML( html_content ))

In [4]:
# Show it as HTML Image (Using Base64)
print_html( seats.to_html_img() )

In [5]:
# Show it as HTML SVG (Native)
print_html( seats.to_svg() )